In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep


In [2]:
# connexion à la première page
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"

# Url ancien
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?nbp=0&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&at=-4&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&ddlFiltres=nofilter"


In [3]:
scrap_list = [[] for x in range(6)]
agence_list =[]
def normalize_text(text):
    temp = str(text).replace("\t","")
    temp = str(temp).replace("\n","")
    temp = str(temp).replace("\r"," ")
    return(temp)

In [4]:
def get_data(url, scrap_list): 
    root_url ="https://www.paruvendu.fr"
    r = requests.get(url)
    if r.connection :
        soup = BeautifulSoup(r.text,"lxml")
        
        h3 = soup.find_all("div", {"class":"ergov3-h3"})
        for elem in h3:     
            scrap_list[0].append(normalize_text(elem.span.text))
            scrap_list[1].append(normalize_text(elem.cite.text))
        
        priceannonce = soup.find_all("div", {"class":"ergov3-priceannonce"})
        for elem in priceannonce: 
            scrap_list[2].append(normalize_text(elem.text))
        
        textannonce = soup.find_all("p", {"class":"txt-long"})
        for elem in textannonce: 
            scrap_list[3].append(normalize_text(elem.text))
        

        lienpro = soup.find_all("div", {"class":"enslogoname_2lines"})
        for elem in lienpro: 
            scrap_list[4].append(normalize_text(elem.span.text))

        agencepro = soup.find_all("div", {"class":"ergov3-lienpro_2lines"})
        for elem in agencepro: 
            try :
                agence_list.append(normalize_text(elem.span.text))
            except AttributeError : 
                agence_list.append("PARTICULIER")
        
        link_url = soup.find_all("a", {"class":"voirann"})
        for elem in link_url:
            if ((str(elem["href"]).startswith("#")) or (str(elem["href"]).startswith("j"))):
                continue
            else :
                temp_url = root_url+str(elem["href"])
                scrap_list[5].append(temp_url)

        
    return(scrap_list)



In [5]:
r = requests.get(url)
soup = BeautifulSoup(r.text,"lxml")
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?nbp=0&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&at=-4&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&ddlFiltres=nofilter"
driver = webdriver.Firefox(executable_path=r'geckodriver.exe')
driver.get(url)
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div/div[1]/div[3]/button[2]'))).click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="batchsdk-ui-alert__buttons_negative"]'))).click()

for i in range(100): 
    sleep(0.1)
    end_url="&p="+str(i+1)
    if i+1 == 1 :
        scrap_list =get_data(url, scrap_list)
    else :
        new_url = url+end_url
        scrap_list =get_data(new_url, scrap_list)





C:\Users\gkerd\AppData\Local\Temp\ipykernel_26780\176678893.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'geckodriver.exe')


In [6]:
import pandas as pd
scrap_df = pd.DataFrame(columns= ["titre_annonce","ville_cp","prix","texte_annonce","type_vendeur","url"])
for i,v in enumerate(scrap_df.columns):
    scrap_df[v] = scrap_list[i]

scrap_df = scrap_df[~scrap_df["titre_annonce"].str.contains("Vente")]
scrap_df["nom_agence"] = agence_list

In [7]:
def format_scrap(df): 
    for column in df.columns :
        df[column] = df[column].str.strip()
    df["ville_cp"] = df["ville_cp"].str.replace("([A-Za-z])(\s)([A-Za-z])", r"\1-\3", regex=True)
    df["code_postal"] = df['ville_cp'].str.split(" ").str[1]
    df['ville'] = df['ville_cp'].str.split(" ").str[0] 
    df["code_postal"] = df["code_postal"].str.replace("[()]","", regex=True)
    df['type'] = df['titre_annonce'].str.split(",").str[0]
    df['surface'] = df['titre_annonce'].str.split(",").str[1]
    df['nb_pieces'] = df['type'].str.extract('(\d+)')
    df['type'] = df['type'].str.split('(\d+)', regex = True).str[0]
    df['surface'] = df['surface'].str.split(' ').str[0]
    df["code_postal"].loc[df["ville"] == "Paris"] = df["code_postal"].loc[df["ville"] == "Paris"].apply(lambda x: "75"+"0"*(3-len(x))+str(x))
    df['departement'] = df["code_postal"].str[:2]
    df["prix"] = df["prix"].str.replace("[*€\s]","", regex=True)
    df.dropna(inplace= True)
    df.drop(df[df.code_postal == ""].index, inplace=True)
    df.reset_index(drop =True, inplace=True)
    df["id"] = df.index
    df= df[["id","type","nb_pieces","surface","prix","ville","code_postal","departement","type_vendeur","texte_annonce","url","nom_agence"]]
    return df



def cast_type(df): 
    for elem in ["nb_pieces","surface","prix","code_postal","departement"] :
        df[elem] = df[elem].astype(int)
    return df

In [8]:
scrap_df = format_scrap(scrap_df)
scrap_df = cast_type(scrap_df)

In [9]:
scrap_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2494 non-null   int64 
 1   type           2494 non-null   object
 2   nb_pieces      2494 non-null   int32 
 3   surface        2494 non-null   int32 
 4   prix           2494 non-null   int32 
 5   ville          2494 non-null   object
 6   code_postal    2494 non-null   int32 
 7   departement    2494 non-null   int32 
 8   type_vendeur   2494 non-null   object
 9   texte_annonce  2494 non-null   object
 10  url            2494 non-null   object
 11  nom_agence     2494 non-null   object
dtypes: int32(5), int64(1), object(6)
memory usage: 185.2+ KB
